In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import scattertext as st
import re, io
from pprint import pprint
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
from spacy.lemmatizer import Lemmatizer
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import gensim.corpora as corpora
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import os, pkgutil, json, urllib
from urllib.request import urlopen
from gensim.models import Word2Vec
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
from gensim.models import Phrases
import codecs
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from sklearn.decomposition import PCA
from gensim.models.word2vec import LineSentence
import gensim
display(HTML("<style>.container { width:98% !important; }</style>"))

/Users/bassim/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [3]:
data = pd.read_csv('reviews.csv')
data = data[['reviews','stars']]
data['sentiment'] = ['positive' if x in [4,5] else 'negative' for x in data['stars']]

In [4]:
nlp = spacy.load('en')

In [5]:
data['first_process'] = [gensim.utils.simple_preprocess(str(x), deacc=True) for x in data['reviews']] 

In [6]:
sentences = []
for sentence in data['first_process']:
    sample = []
    for word in sentence:
        if word not in stop_words:
            sample.append(wordnet_lemmatizer.lemmatize(word))
    sentences.append(sample)


In [7]:
bigram = gensim.models.Phrases([x for x in sentences], min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[sentences], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [8]:
data_lemmatized = [trigram_mod[bigram_mod[sentences[x]]] for x in range(len(sentences))]

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [10]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [11]:
# Build LDA model. takes a little time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=99,
                                           update_every=1,
                                           chunksize=200,
                                           passes=40,
                                            minimum_probability = 0.4,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.345545 -0.180525       1        1  41.898521
5      0.238130  0.013790       2        1  18.070616
3      0.064686  0.410215       3        1  17.187174
1     -0.135859 -0.076185       4        1   4.985686
7     -0.136319 -0.071771       5        1   4.778003
2     -0.140470 -0.050994       6        1   4.671871
0     -0.108499 -0.021448       7        1   4.459434
6     -0.127214 -0.023083       8        1   3.948695, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
430   Default  1198.000000        well  1198.000000  30.0000  30.0000
79    Default  4045.000000     service  4045.000000  29.0000  29.0000
170   Default  1960.000000         day  1960.000000  28.0000  28.0000
38    Default  1704.000000       order  1704.000000  27.0000  27.0000
57    Default  4425.000000         wex  4425.000000  26.0000  26.0000
451   Default  1418.000000        lens  1418.000000  25.0000  25.0000
28    Default  1347.000000        item  1347.000000  24.0000  24.0000
14    Default  1374.000000      camera  1374.000000  23.0000  23.0000
20    Default  2711.000000    delivery  2711.000000  22.0000  22.0000
39    Default  1207.000000     ordered  1207.000000  21.0000  21.0000
4     Default  1061.000000     arrived  1061.000000  20.0000  20.0000
63    Default  2061.000000        good  2061.000000  19.0000  19.0000
35    Default   942.000000        next   942.000000  18.0000  18.0000
483   Default  1789.000000       price  1789.000000  17.0000  17.0000
98    Default  1782.000000      always  1782.000000  16.0000  16.0000
3     Default   454.000000      advice   454.000000  15.0000  15.0000
51    Default  1590.000000        time  1590.000000  14.0000  14.0000
1118  Default  1577.000000   excellent  1577.000000  13.0000  13.0000
1551  Default   440.000000    packaged   440.000000  12.0000  12.0000
387   Default   770.000000         new   770.000000  11.0000  11.0000
297   Default   744.000000   delivered   744.000000  10.0000  10.0000
867   Default  1432.000000       great  1432.000000   9.0000   9.0000
309   Default   683.000000       stock   683.000000   8.0000   8.0000
12    Default   657.000000      bought   657.000000   7.0000   7.0000
236   Default   355.000000   efficient   355.000000   6.0000   6.0000
672   Default   353.000000     quality   353.000000   5.0000   5.0000
1696  Default   307.000000   condition   307.000000   4.0000   4.0000
44    Default  1099.000000     product  1099.000000   3.0000   3.0000
413   Default   306.000000      packed   306.000000   2.0000   2.0000
109   Default  1052.000000    customer  1052.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
832    Topic8   222.958542        help   223.609879   3.2289  -3.2796
476    Topic8   134.718887       asked   135.371307   3.2270  -3.7834
1673   Topic8   132.721146    question   133.372345   3.2269  -3.7983
1266   Topic8   115.440689  despatched   116.095482   3.2261  -3.9378
1390   Topic8   106.594452  additional   107.248299   3.2257  -4.0175
2634   Topic8   100.171501    answered   100.823082   3.2253  -4.0797
992    Topic8    92.518188         pro    93.170357   3.2248  -4.1592
268    Topic8    92.535133        card    93.188042   3.2248  -4.1590
381    Topic8    87.662819     dealing    88.314255   3.2244  -4.2131
1276   Topic8    85.112198       query    85.763565   3.2242  -4.2426
168    Topic8    65.197136     confirm    65.849625   3.2218  -4.5091
1520   Topic8    63.311920       track    63.966736   3.2215  -4.5385
871    Topic8    59.005333        pick    59.662018   3.2207  -4.6089
1337   Topic8    57.385071       speed    58.036732   3.2205  -4.6368
1451   Topic8    54.127720   seriously    54.782280   3.2198  -4.6952
1413   Topic8    52.208340         man    52.861992   3.2